In [10]:
import numpy as np
import copy
import h5py
import matplotlib.pyplot as plt
from PIL import Image
from scipy import ndimage  # PIL and scipy are used here to test the model with my own picture at the end.
from lr_utils import load_dataset

In [11]:
#Loading the data (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()